In [ ]:
import numpy as np

##################################################
##### Matplotlib boilerplate for consistency #####
##################################################
from ipywidgets import interact
from ipywidgets import FloatSlider
from matplotlib import pyplot as plt

%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

global_fig_width = 8
global_fig_height = global_fig_width / 1.61803399
font_size = 12

plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = '0.8'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelpad'] = 8
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlepad'] = 16.0
plt.rcParams['axes.titlesize'] = font_size * 1.4
plt.rcParams['figure.figsize'] = (global_fig_width, global_fig_height)
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif', 'DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = font_size
plt.rcParams['grid.color'] = '0.8'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['grid.linewidth'] = 2
plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dashed_pattern'] = [1, 4]
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['xtick.major.pad'] = 4
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['ytick.major.pad'] = 4
plt.rcParams['ytick.major.size'] = 0
##################################################

## Generating independent samples: sine curve

**Question:** how can we generate independent samples from the following (un-normalised) PDF?

In [ ]:
def show_pdf():
    x = np.linspace(0,4*np.pi,100)
    y = 0.5*(1 + np.sin(2*(x-np.pi/4)))
    plt.plot(x,y)
    plt.fill_between(x,y,alpha=0.2)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$P(x)$')
    plt.show()

In [ ]:
show_pdf()

**Answer:** do the following a large number of times:
1. Generate $x$ coordinates: uniformly-distributed points from $(0, 4\pi)$; where $4\pi$ is the domain of the function.
2. Generate $y$ coordinates: uniformly-distributed points from $(0, 1)$; where 1 is the maximum value of the function.
3. If $y < P(x)$, then accept $x$ coordinate as a sample.
4. If $y \ge P(x)$, then reject $x$ coordinate as a sample.
Known as **Rejection** sampling.

In [ ]:
def P(x):
    return 0.5*(1 + np.sin(2*(x-np.pi/4)))

N = 100000
x = np.random.uniform(0,4*np.pi,N)
y = np.random.uniform(0,1,N)
x_samples = x[y < P(x)]

In [ ]:
plt.hist(x_samples,bins=100,density=True)
plt.show()

## Why do sampling in the first place?

Samples from the posterior:

$$P(\theta | data) = \frac{P(data|\theta) P(\theta)}{P(data)}$$

- Shows the likely *distribution* of parameters, rather than single most likely point. Shows:
 - Correlated parameters
 - Unidentifiable parameters
 - Confidence intervals

## Why do sampling in the first place?

- Often, want to calculate the posterior mean of some parameter, $\theta_1$:
    
$$
E(\theta_1|X) = \int_{\theta_1} \int_{\theta_{-1}} 
\theta_1  P(\theta_1, \theta_{-1} | X)d \theta_{-1} d \theta_1
$$

where $\theta_{-1}$ corresponds to the $d − 1$ other parameters of the
model.

- Posterior mean can be approximated by summing over posterior samples

- Marginal distributions can also be obtained from posterior samples:

$$
P(\theta_1|X) = \int_{\theta_{-1}} P(\theta_1, \theta_{-1} | X)d \theta_{-1}
$$

## Why is generating independent samples difficult?

- **Rejection sampling** requires generation of a large number
of random points to produce relatively few samples.
- This inefficiency increases (exponentially) with the
dimensionality of the distribution; i.e. for posteriors with
more parameters.
- Other methods exist (inverse-transform sampling and
importance sampling, for example) but they suffer from
complexity and/or inefficiency issues.
- We cannot calculate the denominator so are unable to use
some of these methods.
- Even if we had the denominator the complexity of most
models means that independent sampling isn’t possible.

## Is sampling finished?

<center>
<img src="fig/sad-clown.jpg"> 
</center>

## What is dependent sampling?

- A sampling algorithm where the next sample depends on the current value, and the list of all (accepted) positions of the
sampler form the sample.

**Example dependent sampler:** 

choose a new
position based on a local “jumping” distribution
Suppose the next value of the sampler is drawn from a 2d
normal distribution centred on our current position.

In [ ]:
# show dependent sampler here?

## Dependent samplers as Markov Chains (Monte Carlo)

- Where to step next is determined via a distribution
conditional on the current parameter value.
- This stepping is probabilistic $\rightarrow$ *Monte Carlo*.
- The conditional distribution only depends on the current
value of the sampler meaning it is memoryless about past
path.
- This memoryless means that the path of the sampler is a
*1st order Markov Chain*.

## Open questions

How can we decide on the:
1. Starting position.
2. Jumping distribution’s shape.

To ensure convergence to the posterior distribution? Especially
because we cannot compute the posterior itself!

# Random Walk Metropolis

A discrete example: **David Robinson’s fishing**
    
- David Robinson (a more fortunate cousin of Robinson
Crusoe) is marooned on an island.
- Access to four freshwater lakes of different sizes; each with
a supply of fish.

<center>
<img src="fig/david_robinson.svg"> 
</center>

## David Robinson’s fishing

- He has a terrible memory (too much coconut toddy), and
each day forgets any estimates of lake size he made
previously.
- He wants to fish (at maximum) one new lake per day.
- He possesses a coin and a solar-powered calculator that
can generate (pseudo-)random numbers uniformly
distributed between 0 and 1.
- He is initially “washed up” next to lake A.

<center>
<img src="fig/david_robinson.svg"> 
</center>

**Question:** What strategy should he use to fish as
sustainably as possible?

**Remember:** Robinson doesn’t know the # of lakes, nor
the amount of fish in each!

<center>
<img src="fig/david_robinson.svg"> 
</center>

**Answer:** visit each lake in proportion to the fish it contains, by
doing the following:
1. Each night he flips the coin.
2. If it’s heads (tails) he proposes a move to the neighbouring
lake in the clockwise (anticlockwise) direction.
3. Calculates the ratio of the size of the proposed lake to the
current one.
4. Compares the ratio with a (pseudo-)random number from
the calculator.
5. If the ratio exceeds the generated number, he moves. If
not, he stays put and fishes the same lake tomorrow.

## David Robinson’s fishing: does it work?

In [ ]:

from matplotlib import animation
from IPython.display import HTML

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8/1.618))
N = 100
x = [0, 1, 2, 3]
lakes = [2.0, 1.0, 4.0, 2.0]
days = [0.0, 0.0, 0.0, 0.0]
current_lake = 0

def plot():
    ax1.clear()
    days_dots, = ax1.plot(x, days, 'bo', ms=8, label='poisson pmf')
    days_lines = ax1.vlines(x, 0, days, colors='b', lw=5, alpha=0.5)
    ax1.set_ylim(0,N/2)
    return (days_dots, days_lines)

lake_dots, = ax2.plot(x, lakes, 'bo', ms=8, label='poisson pmf')
lake_lines = ax2.vlines(x, 0, lakes, colors='b', lw=5, alpha=0.5)
plot()

def init():
    pl = plot()
    return pl
    
def animate(i):
    current_lake = 1
    proposed_lake = current_lake + int(np.random.uniform() < 0.5)
    ratio = lakes[proposed_lake]/lakes[current_lake]
    if ratio > np.random.uniform():
        current_lake = proposed_lake
    days[current_lake] += 1
    
    pl = plot()
    return pl
    #days_dots.set_data(x,days)

    #days_lines.set_data(x,days)
    
    
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(f, animate,init_func=init,
                               frames=N, interval=20, blit=True)

# call our new function to display the animation
HTML(anim.to_jshtml())